In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [2]:
'''
STEP 1: LOADING DATASET
'''
data_path = '/nfs/gns/literature/Santosh_Tirunagari/GitHub/learning_datasets/'
train_dataset = dsets.MNIST(root=data_path, 
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root=data_path, 
                           train=False, 
                           transform=transforms.ToTensor())

0it [00:00, ?it/s]

9920512it [00:04, 2010479.38it/s]                             


Extracting /nfs/gns/literature/Santosh_Tirunagari/GitHub/learning_datasets/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 118818.23it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /nfs/gns/literature/Santosh_Tirunagari/GitHub/learning_datasets/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:01, 1086183.70it/s]                            
0it [00:00, ?it/s]

Extracting /nfs/gns/literature/Santosh_Tirunagari/GitHub/learning_datasets/MNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 44784.20it/s]            


Extracting /nfs/gns/literature/Santosh_Tirunagari/GitHub/learning_datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [10]:
'''
STEP 2: MAKING DATASET ITERABLE
'''

batch_size = 1000
n_iters = 30000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [11]:
'''
STEP 3: CREATE MODEL CLASS
'''
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [12]:

'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
output_dim = 10

model = LogisticRegressionModel(input_dim, output_dim)

#######################
#  USE GPU FOR MODEL  #
#######################

if torch.cuda.is_available():
    model.cuda()

In [13]:
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()


In [14]:
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.001

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
'''
STEP 7: TRAIN THE MODEL
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        #######################
        #  USE GPU FOR MODEL  #
        #######################
        if torch.cuda.is_available():
            images = Variable(images.view(-1, 28*28).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, 28*28))
            labels = Variable(labels)
        
        # Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        # Forward pass to get output/logits
        outputs = model(images)
        
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # Getting gradients w.r.t. parameters
        loss.backward()
        
        # Updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in test_loader:
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                images = Variable(images.view(-1, 28*28).cuda())
                
                # Forward pass only to get logits/output
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #######################
                #  USE GPU FOR MODEL  #
                #######################
                # Total correct predictions
                correct += (predicted.cpu() == labels.cpu()).sum()
            
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))

Iteration: 500. Loss: 1.8781546354293823. Accuracy: 70
Iteration: 1000. Loss: 1.5917553901672363. Accuracy: 77
Iteration: 1500. Loss: 1.3258112668991089. Accuracy: 80
Iteration: 2000. Loss: 1.1923649311065674. Accuracy: 81
Iteration: 2500. Loss: 1.1072323322296143. Accuracy: 82
Iteration: 3000. Loss: 1.0015507936477661. Accuracy: 82
Iteration: 3500. Loss: 0.9187393188476562. Accuracy: 83
Iteration: 4000. Loss: 0.905571460723877. Accuracy: 83
Iteration: 4500. Loss: 0.834633469581604. Accuracy: 84
Iteration: 5000. Loss: 0.8056172132492065. Accuracy: 84
Iteration: 5500. Loss: 0.7696231007575989. Accuracy: 85
Iteration: 6000. Loss: 0.7239028215408325. Accuracy: 85
Iteration: 6500. Loss: 0.7231870889663696. Accuracy: 85
Iteration: 7000. Loss: 0.6778650879859924. Accuracy: 85
Iteration: 7500. Loss: 0.6815451383590698. Accuracy: 86
Iteration: 8000. Loss: 0.6746867299079895. Accuracy: 86
Iteration: 8500. Loss: 0.6624687910079956. Accuracy: 86
Iteration: 9000. Loss: 0.6437838077545166. Accuracy